# 5.6 - Utilizing Microsoft Azure's Cognitive Services for Text Analysis

This notebook serves to utilize Azure's Cognitive Services in order to apply sentimental analysis on reviews in order to acquire the positiveness confidence scores for each review.

First, install Azure's text analytics library for python:

In [1]:
! pip install azure-ai-textanalytics==5.1.0

## Client Authentication 
In this section, access to the Azure Cognitive Services is granted by providing the Key and Endpoint supplied. 
The Key value is hidden from the code.

In [2]:
key = ""
endpoint = "https://azure-language.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import joblib as jl
import numpy as np

def authenticate_client():
    """
    Client authentication method.

    :return: the authenticated client instance.
    
    """
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

## Sentiment Analysis
After being granted access to Azure's API, we can proceed to load the reviews to be analyzed by Azure's Cognitive Services. 

In [ ]:
def comment_sentiment_analysis(client, documents):
    """
    Method that applies text analysis on the 'documents' argument using the provided 'client'.
    More specifically, store and return all positive confidence scores for all elements in
    'documents'.
    
    :param client: the provided client instance to utilize Azure's API.
    :param documents: a list of strings to be analyzed.
    :return: a list of positive confidence scores.
    
    """
    positiveness = []
    response = client.analyze_sentiment(documents=documents)
    for r in response:
        positiveness.append(r.confidence_scores.positive)
    
    return positiveness

# Load the review comments to be analyzed.
reviews = jl.load('fastapi/data/reviews_most_recent.joblib')
comments = reviews["concat_comments"]

# Due to Cognitive Services restriction of maximum 10 elements to be processed each time,
# the comments are broken into smaller chunks of data.
sentiment = []
for i in range(0, len(comments), 10):
    com = comments.tolist()[i:i+10]
    res = comment_sentiment_analysis(client, com)
    sentiment.append(res)

Last, the unravelled positiveness scores are stored as a binary file.

In [ ]:
jl.dump(np.ravel(sentiment),'fastapi/data/sentiment.joblib')